In [2]:
#Importing the necessary libraries
import pandas as pd
import numpy as np


In [3]:
# Getting the data
#!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')


In [4]:
# data preprocessing block
def preprocess(data):
    # Convert column names to lowercase and replace spaces with underscores
    data.columns = data.columns.str.lower().str.replace(' ', '_')
    
    # Identify categorical columns (object dtype)
    categorical_columns = list(data.dtypes[data.dtypes == 'object'].index)
    
    # Convert string values in categorical columns to lowercase and replace spaces with underscores
    for c in categorical_columns:
        data[c] = data[c].str.lower().str.replace(' ', '_')
    
    return data

In [5]:
preprocess(df)
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)
df.churn = (df.churn == 'yes').astype(int)

# Set up validation data frame
from sklearn.model_selection import train_test_split
df_fulltrain,df_test = train_test_split(df,test_size=0.2,random_state=1)

# Further splitting of training and val data 
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=1)

In [6]:
# Resetting index of the 3 datasets 
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [24]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [7]:
# Setting the y values.
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
# Prepare the  data in dict records

dicts_full_train = df_fulltrain[categorical+numerical].to_dict(orient ='records')
# transform the data to dv
x_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_fulltrain.churn.values

model = LogisticRegression(solver='lbfgs')
model.fit(x_full_train,y_full_train)

In [35]:
# Preparation of test data to be vectorized
dict_test = df_test[categorical+numerical].to_dict(orient ='records')
X_test = dv.transform(dict_test)

In [36]:
# Predictions from X_test
y_pred = model.predict_proba(X_test)[:, 1]